# `Credit Card Exited Prediction`

## Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
tf.__version__

'2.10.1'

## `1) Data Preprocessing`

### Importing Dataset

In [3]:
dataset = pd.read_csv("Churn_Modelling.csv")

In [4]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Removing Unnecessary Columns

In [6]:
dataset.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

### Encoding Categorical Data
Using get_dummies() function

In [7]:
gender = pd.get_dummies(dataset["Gender"], drop_first=True)
geography = pd.get_dummies(dataset["Geography"], drop_first=True)

dataset.drop(["Gender", "Geography"], axis=1, inplace=True)

dataset = pd.concat([dataset, gender, geography], axis=1)

### Splitting Dataset into X and y

In [8]:
X = dataset.drop("Exited", axis=1)
y = dataset["Exited"]

In [9]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Male,Germany,Spain
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1


In [10]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

### Splitting Dataset into Training and Test Set

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=0)

### Feature Scaling
Important Note: Feature Scaling is a **must** for Deep Learning.  
It is not mandatory for ANN, but it is a **good practice** to do it.  
It will help us to **converge faster** to the optimal weights.

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## `2) Building ANN`

### Initializing ANN

In [13]:
ann = tf.keras.models.Sequential() # Sequential is used to initialize the ANN

### Adding Input Layer and First Hidden Layer  
Dense is used to add the input layer and the first hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6,
                              activation="relu")) # relu: rectified linear activation unit

### Adding Second Hidden Layer

In [15]:
ann.add(tf.keras.layers.Dense(units=6,
                              activation="relu"))

### Adding Output Layer  
**Single Neuron** and **sigmoid** activation function in the output layer  
because we have a **binary outcome**.

In [16]:
ann.add(tf.keras.layers.Dense(units=1,
                              activation="sigmoid")) # 0 or 1 (Exited or not)

### Compiling ANN

In [17]:
ann.compile(optimizer="adam", # adam is a stochastic gradient descent algorithm
            loss="binary_crossentropy", # binary_crossentropy is used for binary classification
            metrics=["accuracy"]) # accuracy is used to evaluate the model

## `3) Training ANN`

### Training ANN on Training Set

In [18]:
ann.fit(X_train, y_train,
        batch_size=32, # batch_size: no. of observations weights are updated
        epochs=100) # epochs: no. of times the whole training set is passed through the ANN

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.6275 - accuracy: 0.7084
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4951 - accuracy: 0.8074
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4448 - accuracy: 0.8131
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4291 - accuracy: 0.8163
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4176 - accuracy: 0.8226
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4063 - accuracy: 0.8296
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3953 - accuracy: 0.8359
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3851 - accuracy: 0.8435
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3762 - accuracy: 0.8494
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3691 - accura

## `4) Predictions and Evaluation`

### Predicting for Test Set
predict() returns the probability of the person leaving the bank

In [19]:
predictions = ann.predict(X_test)
predictions = (predictions > 0.5)

63/63 [==============================] - 0s 1ms/step


### Evaluating the Model

In [20]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))

[[1527   68]
 [ 202  203]]
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1595
           1       0.75      0.50      0.60       405

    accuracy                           0.86      2000
   macro avg       0.82      0.73      0.76      2000
weighted avg       0.86      0.86      0.85      2000

0.865
